In [1]:
!nvidia-smi

Mon Dec  6 23:45:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   52C    P8    N/A /  N/A |    711MiB /  4040MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch_geometric
from torch_geometric.datasets import ModelNet
from torch_geometric.transforms import SamplePoints
from torch_geometric.transforms import Compose
from torch_geometric.transforms import LinearTransformation
from torch_geometric.transforms import GenerateMeshNormals
from torch_geometric.transforms import NormalizeScale
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch
from torch_scatter import scatter_mean
import sys

## NOTE:
# Made it work with ModelNet10 (in folder "data/ModelNet") that has only 10 classes
# Now I'll try to test it with ModelNet40 ("data/ModelNet40") with 40 classes

# ----------------------------------------------------------------
# Hyper parameters:
num_points = 1024    # 1024 seems to be the limit...?
batch_size = 32      # not yet used
num_epochs = 100
learning_rate = 0.001
modelnet_num = 10    # 10 or 40


# ----------------------------------------------------------------
# Choosing device:
device = "cuda" if torch.cuda.is_available() else "cpu"


##########################################################################
#                  NO LONGER USED but leave them here...
F = [128, 512, 1024]  # Number of graph convolutional filters.
K = [6, 5, 3]         # Polynomial orders.
M = [512, 128, 10]    # Output dimensionality of fully connected layers.
##########################################################################

# ------------------------------------------------------------------------
## Data loading:
# For ModelNet10 change root to "data/ModelNet"   -> 10 classes
# For MpdelNet40 change root to "data/ModelNet40" -> 40 classes
# Don't forget to change accordingly the output layer from the model...

# Defining the transforms
transforms = Compose([SamplePoints(num_points, include_normals=True), NormalizeScale()])

root = "data/ModelNet"+str(modelnet_num)
dataset_train = ModelNet(root=root, name=str(modelnet_num), train=True, transform=transforms)
dataset_test = ModelNet(root=root, name=str(modelnet_num), train=False, transform=transforms)

# Shuffle Data
dataset_train = dataset_train.shuffle()
dataset_test = dataset_test.shuffle()

# Verification...
print(f"Train dataset shape: {dataset_train}")
print(f"Test dataset shape:  {dataset_test}")

print(dataset_train[0])

In [ ]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv

# taken from rgcn in tensorflow

class GetGraph(nn.Module):
    def __init__(self):
        """
        Creates the weighted adjacency matrix 'W'
        Taked directly from RGCNN
        """
        super(GetGraph, self).__init__()

    def forward(self, point_cloud):
        point_cloud_transpose = point_cloud.permute(0, 2, 1)
        point_cloud_inner = torch.matmul(point_cloud, point_cloud_transpose)
        point_cloud_inner = -2 * point_cloud_inner
        point_cloud_square = torch.sum(torch.mul(point_cloud, point_cloud), dim=2, keepdim=True)
        point_cloud_square_tranpose = point_cloud_square.permute(0, 2, 1)
        adj_matrix = point_cloud_square + point_cloud_inner + point_cloud_square_tranpose
        adj_matrix = torch.exp(-adj_matrix)
        return adj_matrix


class GetLaplacian(nn.Module):
    def __init__(self, normalize=True):
        """
        Computes the Graph Laplacian from a Weighted Graph
        Taken directly from RGCNN - currently not used - might need to find alternatives in PyG for loss function
        """
        super(GetLaplacian, self).__init__()
        self.normalize = normalize

        def diag(self, mat):
        # input is batch x vertices
            d = []
            for vec in mat:
                d.append(torch.diag(vec))
            return torch.stack(d)

    def forward(self, adj_matrix):
        if self.normalize:
            D = torch.sum(adj_matrix, dim=1)
            eye = torch.ones_like(D)
            eye = self.diag(eye)
            D = 1 / torch.sqrt(D)
            D = self.diag(D)
            L = eye - torch.matmul(torch.matmul(D, adj_matrix), D)
        else:
            D = torch.sum(adj_matrix, dim=1)
            D = torch.diag(D)
            L = D - adj_matrix
        return L

In [ ]:
import torch.nn as nn
import torch_geometric.utils as utils
import torch_geometric.nn.conv as conv
import torch

class RGCNN_model(nn.Module):
    def __init__(self, vertice, F, K, M, regularization = 0, dropout = 0):
        # verify the consistency w.r.t. the number of layers
        assert len(F) == len(K)
        super(RGCNN_model, self).__init__()
        '''
        F := List of Convolutional Layers dimensions
        K := List of Chebyshev polynomial degrees
        M := List of Fully Connected Layers dimenstions
        
        Currently the dimensions are 'hardcoded'
        '''
        self.F = F
        self.K = K
        self.M = M

        self.vertice = vertice
        self.regularization = regularization    # gamma from the paper: 10^-9
        self.dropout = dropout
        self.regularizers = []

        # initialize the model layers
        self.get_graph = GetGraph()
        # self.get_laplacian = GetLaplacian(normalize=True)
        self.pool = nn.MaxPool1d(self.vertice)
        self.relu = nn.ReLU()
        self.dropout = torch.nn.Dropout(p=self.dropout)

        ###################################################################
        #                               CHANGE HERE
        self.conv1 = conv.ChebConv(6, 128, 6)
        self.conv2 = conv.ChebConv(128, 512, 5)
        self.conv3 = conv.ChebConv(512, 1024, 3)

        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, modelnet_num)
        ###################################################################

        '''
        for i in range(len(F)):
            if i == 0:
                layer = ChebConv(Fin=3, K=K[i], Fout=F[i])
            else:
                layer = ChebConv(Fin=F[i-1], K=K[i], Fout=F[i])
            setattr(self, 'gcn%d'%i, layer)
        for i in range(len(M)):
            if i==0:
                layer = nn.Linear(F[-1], M[i])
            else:
                layer = nn.Linear(M[i-1], M[i])
            setattr(self, 'fc%d'%i, layer)
        '''

    def forward(self, x):
        self.regularizers = []
        # forward pass
        W   = self.get_graph(x.detach())  # we don't want to compute gradients when building the graph
        edge_index, edge_weight = utils.dense_to_sparse(W)
        out = self.conv1(x, edge_index, edge_weight)
        out = self.relu(out)
        edge_index, edge_weight = utils.remove_self_loops(edge_index, edge_weight)
        L_edge_index, L_edge_weight = torch_geometric.utils.get_laplacian(edge_index.detach(), edge_weight.detach(), normalization="sym")
        L = torch_geometric.utils.to_dense_adj(edge_index=L_edge_index, edge_attr=L_edge_weight)
        self.regularizers.append(torch.linalg.norm(torch.matmul(torch.matmul(torch.Tensor.permute(out.detach(), [0, 2, 1]), L), out.detach())))

        W   = self.get_graph(out.detach())
        edge_index, edge_weight = utils.dense_to_sparse(W)
        out = self.conv2(out, edge_index, edge_weight)
        out = self.relu(out)
        edge_index, edge_weight = utils.remove_self_loops(edge_index, edge_weight)
        L_edge_index, L_edge_weight = torch_geometric.utils.get_laplacian(edge_index.detach(), edge_weight.detach(), normalization="sym")
        L = torch_geometric.utils.to_dense_adj(edge_index=L_edge_index, edge_attr=L_edge_weight)
        self.regularizers.append(torch.linalg.norm(torch.matmul(torch.matmul(torch.Tensor.permute(out.detach(), [0, 2, 1]), L), out.detach())))

        W   = self.get_graph(out.detach())
        edge_index, edge_weight = utils.dense_to_sparse(W)
        out = self.conv3(out, edge_index, edge_weight)
        out = self.relu(out)
        edge_index, edge_weight = utils.remove_self_loops(edge_index, edge_weight)
        L_edge_index, L_edge_weight = torch_geometric.utils.get_laplacian(edge_index.detach(), edge_weight.detach(), normalization="sym")
        L = torch_geometric.utils.to_dense_adj(edge_index=L_edge_index, edge_attr=L_edge_weight)
        self.regularizers.append(torch.linalg.norm(torch.matmul(torch.matmul(torch.Tensor.permute(out.detach(), [0, 2, 1]), L), out.detach())))

        '''
        for i in range(len(self.F)):
            x = getattr(self, 'gcn%d'%i)(x, L)
            print(x)
            x = self.relu(x)
        '''

        out = out.permute(0, 2, 1) # Transpose
        out = self.pool(out)
        out.squeeze_(2)

        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        for param in self.fc1.parameters():
            self.regularizers.append(torch.linalg.norm(param))

        out = self.fc2(out)
        out = self.relu(out)
        out = self.dropout(out)
        for param in self.fc1.parameters():
            self.regularizers.append(torch.linalg.norm(param))
        out = self.fc3(out)
        for param in self.fc1.parameters():
            self.regularizers.append(torch.linalg.norm(param))
        '''
        for i in range(len(self.M)):
            x = getattr(self, "fc%d"%i)(x)
        return x
        '''

        return out, self.regularizers
'''
model = RGCNN_model(num_points, F, K, M)

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
'''

In [ ]:
# Training
model = RGCNN_model(num_points, F, K, M, dropout=0.6)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss = torch.nn.CrossEntropyLoss()
def get_loss(y, labels, regularization, regularizers):
    cross_entropy_loss = loss(y, labels)
    s = torch.sum(torch.as_tensor(regularizers))
    regularization *= s
    l = cross_entropy_loss + regularization
    return l
    
correct_percentage_list = []
model.train()
for epoch in range(num_epochs):
    #dataset_train = dataset_train.shuffle()
    correct = 0
    for i, data in enumerate(dataset_train):
        # make sure the gradients are empty
        optimizer.zero_grad()
        
        # Data preparation 
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)

        # Forward pass
        y_pred, regularizers = model(x)     # (1 * 40)
        
        class_pred = torch.argmax(y_pred.squeeze(0))  # (1)  
        correct += int((class_pred == y).sum())       # to compute the accuracy for each epoch
        

        # loss and backward
        ###################################################################################
        #                           CrossEntropyLoss
        # This WORKS but I am testing the other way...
        # l = loss(y_pred, y)   # one value
        # l.backward()          # update gradients
        ###################################################################################
         
        # set 'regularization' to 0 to get only the cross-entropy-loss
        l = get_loss(y_pred, y, regularization=1e-9, regularizers=regularizers)
        l.backward()

        # optimisation
        optimizer.step()
    
        if i%100==0:
            print(f"Epoch: {epoch}, Sample: {i}, Loss:{l} - Predicted class vs Real Cass: {class_pred} <-> {y.item()}")
            # print(torch.sum(torch.as_tensor(regularizers)))
    print(f"~~~~~~~~~ CORRECT: {correct / len(dataset_train)} ~~~~~~~~~~~")
    correct_percentage_list.append(correct / len(dataset_train))
print(correct_percentage_list)

torch.save(model.state_dict(), "/home/alex/Alex_pyt_geom/models/model.pt")

with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred, _ = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

In [ ]:
print(correct_percentage_list)

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        # print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")

In [ ]:
with torch.no_grad():
    model.eval()
    correct = 0
    for data in dataset_test:
        
        pos = data.pos        # (num_points * 3)   
        normals = data.normal # (num_points * 3)
        x = torch.cat([pos, normals], dim=1)   # (num_points * 6)
        x = x.unsqueeze(0)    # (1 * num_points * 6)     the first dimension may be used for batching?
        x = x.type(torch.float32)  # other types of data may be unstable

        y = data.y              # (1)
        y = y.type(torch.long)  # required by the loss function
        
        x = x.to(device)      # to CUDA if available
        y = y.to(device)
     
        # Forward pass
        y_pred, _ = model(x)     # (1 * 40)

        class_pred = torch.argmax(y_pred)
        print("Pred: ", class_pred.item(), "Real: " , y.item())
        correct += int((class_pred == y_pred).sum())

    print(f"Correct percentage : {correct / len(dataset_test)}")
    

In [ ]:
with torch.no_grad():
    x = torch.rand([1, 1024, 6])*5+3
    x = x.to(device)
    y = model(x)
    y_class = torch.argmax(y)
    print(y)
    print(y_class)